In [41]:
import numpy as np
print("Loading training data ...")
data_train = np.genfromtxt("data/LSDA2017GalaxiesTrain.csv",delimiter=",", dtype=float)
Xtrain, ytrain = data_train[:,:-1], data_train[:,-1]
print("Loaded training data: n=%i, d=%i" % (Xtrain.shape[0], Xtrain.shape[1]))

data_validation = np.genfromtxt("data/LSDA2017GalaxiesValidate.csv",delimiter=",", dtype=float)
Xvd, yvd = data_validation[:,:-1], data_validation[:,-1]
print("Loaded validation data: n=%i, d=%i" % (Xvd.shape[0], Xvd.shape[1]))

data_test = np.genfromtxt("data/LSDA2017GalaxiesTest.csv",delimiter=",", dtype=float)
Xtest, ytest = data_test[:,:-1], data_test[:,-1]
print("Loaded test data: n=%i, d=%i" % (Xtest.shape[0], Xtest.shape[1]))

Loading training data ...
Loaded training data: n=4000, d=18
Loaded validation data: n=1000, d=18
Loaded test data: n=5000, d=18


In [42]:
ytrain_var = np.std(ytrain)**2
print("Standard deviation for train label is %f" % ytrain_var)

def tf_input(X,y):
    # Preprocess your data here...

    #X = tf.constant(X)
    #y = tf.constant(y)
    return X, y

Standard deviation for train label is 0.010498


In [43]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Splits data into mini-batches
# Batches are not randomized/shuffled, shuffling the data in mini-batch learning typically improves the performance
class Batcher:
    'Splits data into mini-batches'
    def __init__(self, data, batchSize):
        self.data, self.target = data
        self.batchSize = batchSize
        self.batchStartIndex = 0
        self.batchStopIndex = 0
        self.noData = self.data.shape[0]
    def nextBatch(self):
        self.batchStartIndex = self.batchStopIndex % self.noData
        self.batchStopIndex = min(self.batchStartIndex + self.batchSize, self.noData)
        return self.data[self.batchStartIndex:self.batchStopIndex], self.target[self.batchStartIndex:self.batchStopIndex]

# Flags
try:
   FLAGS
except NameError:
    flags = tf.app.flags
    FLAGS = flags.FLAGS
    flags.DEFINE_string('summary_dir', 'tensor_logs', 'directory to put the summary data')
    flags.DEFINE_string('data_dir', 'data', 'directory with data')
    flags.DEFINE_integer('maxIter', 10000, 'number of iterations')
    flags.DEFINE_integer('batchSize', 128, 'batch size')
    flags.DEFINE_integer('noHidden1', 64, 'size of first hidden layer')
    flags.DEFINE_integer('noHidden2', 32, 'size of second hidden layer')
    flags.DEFINE_float('lr', 0.001, 'initial learning rate')

X_train ,y_train = tf_input(Xtrain,ytrain)
# Number of training data points
noTrain = X_train.shape[0]
print("Numer of training data points:", noTrain)

# Input dimension
inDim = X_train.shape[1]

X_vd,y_vd = tf_input(Xvd,yvd)
X_test ,y_test = tf_input(Xtest,ytest)

Numer of training data points: 4000


In [49]:
import os
if not os.path.exists(flags.FLAGS.summary_dir):
    os.makedirs(flags.FLAGS.summary_dir)
import shutil
shutil.rmtree(flags.FLAGS.summary_dir)

# Create graph
sess = tf.Session()

# Initialize placeholders
x_data = tf.placeholder(shape=[None, inDim], dtype=tf.float32, name='input')
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='target')

# Define variables
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=np.std(ytrain)) # restrict to +/- 2*stddev
    return tf.Variable(initial, name='weights')

def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=np.std(ytrain))
    return tf.Variable(initial, name='bias')


# Define model
with tf.name_scope('layer1') as scope:
    W_1 = weight_variable([inDim, FLAGS.noHidden1])
    b_1 = bias_variable([FLAGS.noHidden1])
    y_1 = tf.matmul(x_data , W_1 ) + b_1
    y_1 = tf.nn.relu(y_1)
    

with tf.name_scope('layer2') as scope:
    W_2 = weight_variable([FLAGS.noHidden1, 1])
    b_2 = bias_variable([1])
    model_output = tf.matmul(y_1, W_2) + b_2


# Declare loss function
loss = tf.reduce_mean(tf.square(model_output - y_target), name='mean_squared_error')
tf.summary.scalar('squared_error', loss)

# Declare optimizer
my_opt =  tf.train.GradientDescentOptimizer(FLAGS.lr)
train_step = my_opt.minimize(loss)

# Map model output to validation set
with tf.name_scope('regression_predictions') as scope:
    predictions_correct = tf.abs(model_output - y_target)
accuracy = tf.reduce_mean(predictions_correct, name='mean_accuracy')
tf.summary.scalar('accuracy', accuracy)


# Logging
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summary_dir + '/train')
test_writer = tf.summary.FileWriter(FLAGS.summary_dir + '/test')
validate_writer=  tf.summary.FileWriter(FLAGS.summary_dir + '/validate')
writer = tf.summary.FileWriter(FLAGS.summary_dir, sess.graph)
saver = tf.train.Saver() # for storing the best network



In [50]:
# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# Best validation accuracy seen so far
bestValidation = 0.0

# Mini-batches for training
batcher = Batcher((X_train,y_train), FLAGS.batchSize)

# Training loop
for i in range(FLAGS.maxIter):
    xTrain, yTrain = batcher.nextBatch()
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    sess.run(train_step, feed_dict={x_data: xTrain, y_target: np.transpose([yTrain])})
    summary = sess.run(merged, feed_dict={x_data: xTrain, y_target: np.transpose([yTrain])})
    train_writer.add_summary(summary, i)
    if((i+1)%100==0):
        print("Iteration:",i+1,"/",FLAGS.maxIter)
        summary = sess.run(merged, feed_dict={x_data: X_test, y_target: np.transpose([y_test])})
        test_writer.add_summary(summary, i)
        #currentValidation, summary = sess.run([accuracy, merged], feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])})
        validate_writer.add_summary(summary, i)
        if(currentValidation > bestValidation):
            bestValidation = currentValidation
            saver.save(sess=sess, save_path=FLAGS.summary_dir + '/bestNetwork')
            print("\tbetter network stored,",currentValidation,">",bestValidation)

# Print values after last training step
print("final training accuracy:", sess.run(accuracy, feed_dict={x_data: X_train, y_target: np.transpose([y_train])}),
      "final test accuracy: ", sess.run(accuracy, feed_dict={x_data: X_test, y_target: np.transpose([y_test])}),
      "final validation accuracy: ", sess.run(accuracy, feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])}))

# Load the network with the lowest validation error
saver.restore(sess=sess, save_path=FLAGS.summary_dir + '/bestNetwork')
print("best training accuracy:", sess.run(accuracy, feed_dict={x_data: X_train, y_target: np.transpose([y_train])}),
      "best test accuracy: ", sess.run(accuracy, feed_dict={x_data: X_test, y_target: np.transpose([y_test])}),
      "best validation accuracy: ", sess.run(accuracy, feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])}))


InvalidArgumentError: You must feed a value for placeholder tensor 'input_8' with dtype float
	 [[Node: input_8 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'input_8', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lsda/TensorFlow/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/lsda/TensorFlow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/lsda/TensorFlow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/lsda/TensorFlow/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/lsda/TensorF